# 使用 langchain_glm 框架并利用 glm-4-alltools 模型
**This tutorial is Only in Chinese explanation**

[langchain_glm](https://github.com/MetaGLM/langchain-glm) 是智谱AI 推出的一个工具包，用于更好的利用好 GLM-4 All Tools能力并接入 LangChain框架的工具包。通过langchain的基础组件，实现了完整的支持智能体和相关任务架构。底层采用智谱AI的最新的 GLM-4 All Tools, 通过智谱AI的API接口， 能够自主理解用户的意图，规划复杂的指令，并能够调用一个或多个工具（例如网络浏览器、Python解释器和文本到图像模型）以完成复杂的任务。本教程将带领开发者使用 langchain_glm 框架，实现一个简单的工具链。


## 1. 安装依赖

首先，我们需要安装 langchain_glm 框架，在这里，我们使用pip进行安装。

In [1]:
!pip install langchain_glm langchain

    extract-msg (<=0.29.*)
                 ~~~~~~~^


## 2. 初始化

在这里，我们初始化一个 ChatZhipuAI 对象，用于调用智谱AI的GLM-4模型。您需要在这里填入您的 API_KEY。在这里，您可以选择智谱AI的任何一个模型，例如 glm-4-0520。

In [ ]:
import os

os.environ["ZHIPUAI_API_KEY"] = "your api key"
from langchain_glm import ChatZhipuAI

llm = ChatZhipuAI(model="glm-4-0520")

## 3. 创建工具

在这里，我们创建一些工具，用于后续的工具链调用。这里只使用了最简单的例子，构建了三个工具，分别用于相乘、相加和求幂。

In [3]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base ** exponent

## 4. 创建工具链

在这里，我们将工具链和工具绑定在一起，然后创建一个简单的工具链，用于调用工具。

In [4]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable

tools = [multiply, exponentiate, add]
llm_with_tools = llm.bind_tools(tools)
tool_map = {tool.name: tool for tool in tools}


def call_tools(msg: AIMessage) -> Runnable:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls


chain = llm_with_tools | call_tools

In [5]:
chain.invoke(
    "What's 23 times 7, and what's five times 18 and add a million plus a billion and cube thirty-seven"
)

[{'name': 'multiply',
  'args': {'first_int': 23, 'second_int': 7},
  'id': 'call_20240713191238c7f4b21cd22e4b759710754d08ee755c',
  'output': 161}]

## 5. 使用 langchain_glm 框架

在这里，我们使用 langchain_glm 框架，实现一个简单的工具链。首先，我们需要创建一个智能体执行器，然后调用智能体执行器的invoke方法，传入一个字符串，用于调用智能体。
需要注意的是，在这里，您必须使用 `glm-4-alltools` 模型，因为 langchain_glm 框架只支持 GLM-4 All Tools模型
(有且仅有该模型支持工具构建和自动执行的能力,剩下的模型仅能完成工具调用)。


In [6]:
from langchain_glm.agents.zhipuai_all_tools import ZhipuAIAllToolsRunnable

agent_executor = ZhipuAIAllToolsRunnable.create_agent_executor(
    model_name="glm-4-alltools",
    tools=[
        {"type": "code_interpreter", "code_interpreter": {"sandbox": "none"}},
        {"type": "web_browser"},
        {"type": "drawing_tool"},
        multiply, exponentiate, add
    ],
)


## 6. 调用智能体

模型会根据输入，自行构建和调用工具链，然后返回结果。

In [7]:
from langchain_glm.agents.zhipuai_all_tools.base import (
    AllToolsAction,
    AllToolsActionToolEnd,
    AllToolsActionToolStart,
    AllToolsFinish,
    AllToolsLLMStatus
)
from langchain_glm.callbacks.agent_callback_handler import AgentStatus

chat_iterator = agent_executor.invoke(
    chat_input="看下本地文件有哪些，告诉我你用的是什么文件,查看当前目录"
)
async for item in chat_iterator:
    if isinstance(item, AllToolsAction):
        print("AllToolsAction:" + str(item.to_json()))
    elif isinstance(item, AllToolsFinish):
        print("AllToolsFinish:" + str(item.to_json()))
    elif isinstance(item, AllToolsActionToolStart):
        print("AllToolsActionToolStart:" + str(item.to_json()))
    elif isinstance(item, AllToolsActionToolEnd):
        print("AllToolsActionToolEnd:" + str(item.to_json()))
    elif isinstance(item, AllToolsLLMStatus):
        if item.status == AgentStatus.llm_end:
            print("llm_end:" + item.text)



> Entering new ZhipuAiAllToolsAgentExecutor chain...
llm_end:好的，我将列出当前目录下所有的文件。抱歉，我无法直接查看您本地文件系统的内容。如果您需要查看当前目录下的文件，您可以使用命令行工具，例如在Linux系统中使用`ls`命令，在Windows系统中使用`dir`命令。这些命令会列出当前目录下的所有文件和文件夹。如果您需要进一步的帮助，请提供具体的操作系统信息。
search("当前目录下的文件")
title:在Linux中，如何查看当前目录下的文件和文件夹？ - 黄嘉波 ...
link:https://www.cnblogs.com/huangjiabobk/p/18152538
content:ls 是最基本也是最常用的命令，用于列出当前目录下的文件和文件夹。 基本用法 ： ls. 长格式列出 （显示文件权限、所有者、大小和最后修改时间）： ls -l. 按时间排序 （由新到旧）： ls -lt. 按时间排序 （由旧到新）： ls -ltr. 显示隐藏文件 （以点开头的文件）： ls -a. 2. echo 命令. echo 命令通常用于显示文本，但也可以与文件结合使用来显示当前目录的内容。 列出当前目录的文件和文件夹 ： echo * 3. find 命令. find 命令可以用来查找并显示当前目录及其子目录下的所有文件和文件夹。 列出当前目录的内容 ： find . 4. tree 命令.
title:Linux ls 命令 - 菜鸟教程
link:https://www.runoob.com/linux/linux-comm-ls.html
content:Linux ls （英文全拼： list directory contents）命令用于显示指定工作目录下之内容（列出目前工作目录所含的文件及子目录)。
title:Linux中，显示当前目录位置、列出当前目录下的目录和文件 ...
link:https://blog.csdn.net/qq_46357586/article/details/104565588
content:一.显示当前目录位置. pwd. 二.列出当前目录下的目录和文件. ls --help：得到所有关于ls的命令. 三.切换或进入目录命令. 进入根目录： c